Set up PLGA self-assembly simulations
====

## Libraries

In [1]:
import os
import shutil
import sys

## Create the topology <a name="S1"></a> 

We use [`Packmol`](http://leandro.iqm.unicamp.br/m3g/packmol/userguide.shtml) code to arrange AuNP in the center of the box and to randomly distribute the PLGA molecules in the rest of the volume (13x13x13 $\mathrm{nm^{3}}$).

> **Note**: to consider molecular modeling configurations including one AuNP and a different concentrations of PLGA we have to change `nPLGA` variable. 

> Remember to change the name of the `output` file, to not overwrite them.

In [ ]:
%cd ../simulations/0-Packmol/

file_NP = "AuNP_ASE_wulff_surf_d39.pdb"
file_PLGA = "2PLGA_chain_conect.pdb"
nPLGA = 60

with open(
    os.path.join("inputs_self-assembly", "packmol_%d.in" % nPLGA), "w"
) as file_obj:
    file_obj.write(
        """
filetype pdb
tolerance 2.0
seed 42  

structure {:s}
number 1
center
fixed 65.0 65.0 65.0 0.0 0.0 0.0
end structure 

structure {:s}
number {:d}
inside box 0.0 0.0 0.0 130.0 130.0 130.0 
end structure 

output gold_60plga.pdb     

        """.format(
            file_NP, file_PLGA, nPLGA
        )
    )

In [ ]:
# Change the name of the packmol input

%cd inputs_self-assembly/
!packmol < packmol_60.in

In [ ]:
shutil.copyfile(
    os.path.join(f"gold_{nPLGA}plga.pdb"),
    os.path.join("..", "..", "1-PLGA_self-assembly", f"{nPLGA}", "TOP", "ic.pdb"),
)

In [ ]:
%cd ../../1-PLGA_self-assembly

Then, we generate the topology through `GROMACS` software, using the mixed force field `mix_oplsaa.ff`.

In [ ]:
# Change the folder name and generate the topology for all the configurations

%cd 60/TOP
!gmx pdb2gmx -f ic.pdb -o ic.gro -water spce -ff mix_oplsaa

> Remember to change the name of the position restrains in POSRES_GOLD and in POSRES_PLGA for the files `topol_Other.itp` and `topol_Other_chain_A.itp`, respectively

## Our MD protocol to simulate the self-assembly of 60 PLGA on a AuNP <a name="S2"></a> 

### Define box and carry out solvation <a name="S2.1"></a> 

In [ ]:
%cd ../BOX
!gmx editconf -f ../TOP/ic.gro -o newbox.gro -c no -center 6.5 6.5 6.5 -box 13.0 13.0 13.0 -bt triclinic
!gmx solvate -cp newbox.gro -cs spc216.gro -o solv.gro -p ../TOP/topol.top

### Energy minimization <a name="S2.2"></a> 

In [ ]:
%cd ../EM
!gmx grompp -f minim.mdp -r ../BOX/solv.gro -c ../BOX/solv.gro -p ../TOP/topol.top -o em.tpr  
!gmx mdrun -v -deffnm em					


### Equilibration <a name="S2.3"></a> 

We apply the canonical ensemble `NVT` to reach an equilibrium temperature of 300 K

In [ ]:
%cd ../NVT
!gmx grompp -f nvt.mdp -r ../EM/em.gro -c ../EM/em.gro -p ../TOP/topol.top -o nvt.tpr  
!gmx mdrun -v -deffnm nvt					

We set the isothermal-isobaric ensemble `NPT` at an equilibrium pressure of 1 bar

In [ ]:
%cd ../NPT
!gmx grompp -f npt.mdp -r ../NVT/nvt.gro -c ../NVT/nvt.gro -p ../TOP/topol.top -o npt.tpr  
!gmx mdrun -v -deffnm npt	

### Run MD <a name="S2.4"></a> 

In [ ]:
%cd ../MD
!gmx grompp -f md.mdp -r ../NPT/npt.gro -c ../NPT/npt.gro -t ../NPT/npt.cpt -p ../TOP/topol.top -o md_run.tpr  
!gmx mdrun -v -deffnm md_run 	

In [12]:
%cd ../../

/home/roberta/SM_AuNP_PLGA/simulations/1-PLGA_self-assembly


> **NOTE**: We simulated the other configurations with different concentrations of PLGA following the same MD protocol illustrated. 

> First, remember to perform the following steps in each main folders (`30`, `20` and `10`):

> 1. Create the folders: `!mkdir BOX`, `!mkdir EM`, `!mkdir NVT`, `!mkdir NPT` and `!mkdir MD` 
> 2. Imports the input files (`minim.mdp`, `nvt.mdp`, `npt.mdp` and `md.mdp`) into their respective folders using: `shutil.copyfile(os.path.join(), os.path.join())`